In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import  Select
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
# import numpy as np
import time


position_dict = {'QB': 2, 'RB': 3, 'WR': 4, 'TE': 5}
# week_dict = {"NFL Week 1": 8,"NFL Week 2": 7, "NFL Week 3": 6, "NFL Week 4": 5, "NFL Week 5": 4, "NFL Week 6": 3}
week_dict = { "NFL Week 7": 4}

driver = webdriver.Chrome()
driver.get("https://fantasy.espn.com/football/leaders")

### Take table path and turn it into a Dataframe

In [2]:

def extract_table_to_dataframe(header_xpath: str, tBody_xpath: str) -> pd.DataFrame:
    # Extract the column names from the header row
    header_row = driver.find_element(By.XPATH, header_xpath)
    cols = header_row.find_elements(By.TAG_NAME, 'th')
    column_names = [col.text for col in cols]
    df = pd.DataFrame(columns=column_names)
    tbody = driver.find_element(By.XPATH, tBody_xpath)
    rows = tbody.find_elements(By.TAG_NAME, 'tr')
    for row in rows:
        temp_list = [col.text for col in row.find_elements(By.TAG_NAME, 'td')]
        df.loc[len(df)] = temp_list
    return df

### Loop to Extract data on each page combination

In [3]:

final_df = pd.DataFrame()

for week, week_index in week_dict.items():
    dropdown = driver.find_element(By.XPATH, f'//*[@id="fitt-analytics"]/div/div[5]/div[2]/div[2]/div[1]/div/div[2]/div[5]/div/div[2]/select/option[{week_index}]')
    print(f'Week: {week}')
    dropdown.click()
    time.sleep(3)
    # Loop through positions
    for position, position_index in position_dict.items():
        print(f'Position: {position}')
        label = driver.find_element(By.XPATH, f'//*[@id="filterSlotIds"]/label[{position_index}]')
        label.click()
        while True: 
            time.sleep(3)
            start = time.time()
            header_row = '//*[@id="fitt-analytics"]/div/div[5]/div[2]/div[3]/div/div/div/div/div/div/table[1]/thead/tr[2]'
            tbody = '//*[@id="fitt-analytics"]/div/div[5]/div[2]/div[3]/div/div/div/div/div/div/table[1]/tbody'
            df1=extract_table_to_dataframe(header_xpath=header_row, tBody_xpath=tbody)
            header_row = '//*[@id="fitt-analytics"]/div/div[5]/div[2]/div[3]/div/div/div/div/div/div/div/div[2]/table/thead/tr[2]'
            tbody = '//*[@id="fitt-analytics"]/div/div[5]/div[2]/div[3]/div/div/div/div/div/div/div/div[2]/table/tbody'
            df2=extract_table_to_dataframe(header_xpath=header_row, tBody_xpath=tbody)
            header_row = '//*[@id="fitt-analytics"]/div/div[5]/div[2]/div[3]/div/div/div/div/div/div/table[2]/thead/tr[2]'
            tbody = '//*[@id="fitt-analytics"]/div/div[5]/div[2]/div[3]/div/div/div/div/div/div/table[2]/tbody'
            df3=extract_table_to_dataframe(header_xpath=header_row, tBody_xpath=tbody)  
            df_combined = pd.concat([df1, df2, df3], axis=1)
            df_combined['week'] = str(week)

            frames = [final_df, df_combined]
            final_df = pd.concat(
                frames,
                axis=0,
                join="outer",
                ignore_index=True,

                )
            try: 
                # button = driver.find_element(By.XPATH, f'//*[@id="fitt-analytics"]/div/div[5]/div[2]/div[3]/div/div/div/div/nav/button[2]')
                button = driver.find_element(By.XPATH, f'/html/body/div[1]/div[1]/div/div/div[5]/div[2]/div[3]/div/div/div/div/nav/button[2]')                                     
                if button.is_enabled():
                    button.click()
                    print(f'Next page {time.time() - start}')
                else:
                    print(f'End of Position {time.time() - start}')
                    break
            except Exception as e:
                print(f'Custom Exception {e}')
                break

Week: NFL Week 7
Position: QB
Next page 6.817121982574463
Next page 6.878649950027466
End of Position 4.8633904457092285
Position: RB
Next page 7.254326581954956
Next page 6.543887615203857
Next page 6.595282316207886
Next page 6.3647682666778564
Next page 6.454894542694092
End of Position 3.0267598628997803
Position: WR
Next page 6.854191303253174
Next page 6.438606262207031
Next page 6.842204332351685
Next page 6.530642032623291
Next page 6.356172800064087
Next page 6.6051905155181885
Next page 6.638225317001343
End of Position 5.263035535812378
Position: TE
Next page 6.589824438095093
Next page 6.650367736816406
Next page 6.422682285308838
Next page 6.616470575332642
End of Position 2.1190340518951416


In [4]:
pd.set_option('display.max_rows', 2000)  # Set the maximum number of rows to display
pd.set_option('display.max_columns', 2000)  # Set the maximum number of columns to display
final_df.tail()

,PLAYER,TYPE,ACTION,OPP,STATUS,PROJ,C/A,YDS,TD,INT,CAR,YDS,TD,REC,YDS,TD,TAR,2PC,FUML,TD,FPTS,week
1005,Darren Fells\nFA\nTE,,,--,,--,--/--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,NFL Week 7
1006,Antony Auclair\nFA\nTE,,,--,,--,--/--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,NFL Week 7
1007,Adam Shaheen\nQ\nFA\nTE,,,--,,--,--/--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,NFL Week 7
1008,Luke Stocker\nFA\nTE,,,--,,--,--/--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,NFL Week 7
1009,Hunter Bryant\nQ\nFA\nTE,,,--,,--,--/--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,NFL Week 7


### Modify, Add and Drop Columns

In [5]:
test_df = final_df.copy()


# Create empty columns for Name, Team, and Position
test_df['Name'] = ''
test_df['Team'] = ''
test_df['Position'] = ''

# Iterate through the DataFrame row by row
for index, row in test_df.iterrows():
    parts = row['PLAYER'].split('\n')
    if len(parts) == 4:
        # If there are 4 parts, the format is "Name\nQ/O/IR\nTeam\nPosition"
        test_df.at[index, 'Name'] = parts[0]
        test_df.at[index, 'Team'] = parts[2]
        test_df.at[index, 'Position'] = parts[3]
    elif len(parts) == 3:
        # If there are 3 parts, the format is "Name\nTeam\nPosition"
        test_df.at[index, 'Name'] = parts[0]
        test_df.at[index, 'Team'] = parts[1]
        test_df.at[index, 'Position'] = parts[2]
    

# Create the 'atHome' column based on the 'OPP' column
test_df['atHome'] = ~test_df['OPP'].str.contains('@')
test_df['OPP'] = test_df['OPP'].str.replace('@', '')

# front_columns = ['Name', 'Team', 'Position','OPP', 'atHome']
# test_df = test_df[front_columns + [col for col in test_df if col not in front_columns]]

# Extract the integer part from the 'week' column
test_df['week'] = test_df['week'].str.extract(r'(\d+)').astype(int)

# Split the 'STATUS' column into 'Won' and 'PointsScored'
test_df[['Won', 'PointsScored']] = test_df['STATUS'].str.split(' ', expand=True)
test_df['Won'] = test_df['Won'].map({'W': True, 'L': False})

test_df['PointsScored'] = test_df['PointsScored'].str.split('-').str[0]

test_df = test_df.replace({'--': 0, '--/--': '0/0'})
test_df.rename(columns={'C/A': 'CompletedPasses'}, inplace=True)
test_df['CompletedPasses'] = test_df['CompletedPasses'].str.split('/').str[0].astype(int)


to_drop = ['PLAYER','TYPE','ACTION','STATUS']
test_df.drop(columns=to_drop, inplace=True)

columns_to_move = ['Name', 'Position', 'Team', 'OPP','atHome','Won','PointsScored','PROJ']
test_df = pd.concat([test_df[columns_to_move], test_df.drop(columns=columns_to_move)], axis=1)


test_df.head(5)

,Name,Position,Team,OPP,atHome,Won,PointsScored,PROJ,CompletedPasses,YDS,TD,INT,CAR,YDS,TD,REC,YDS,TD,TAR,2PC,FUML,TD,FPTS,week
0,Lamar Jackson,QB,Bal,Det,True,True,38,19.1,21,357,3,0,9,36,1,0,0,0,0,0,1,0,33.88,7
1,Patrick Mahomes,QB,KC,LAC,True,True,31,22.8,32,424,4,1,4,29,0,0,0,0,0,0,0,0,33.86,7
2,Gardner Minshew,QB,Ind,Cle,True,False,38,11.5,15,305,2,1,3,29,2,0,0,0,0,0,3,0,27.1,7
3,Josh Allen,QB,Buf,NE,False,False,25,19.7,27,265,2,1,7,17,1,0,0,0,0,0,1,0,24.3,7
4,Jalen Hurts,QB,Phi,Mia,True,True,31,23.5,23,279,2,1,11,21,1,0,0,0,0,0,1,0,23.26,7


In [6]:
# Drop rows where 'WON' is NaN
test_df = test_df.dropna(subset=['Won'])
column_mapping = {
    0: 'NAME',
    1: 'POS',
    2: 'TEAM',
    3: 'OPP',
    4: '@HOME',
    5: 'WON',
    6: 'TEAM_PTS',
    7: 'PROJ_PTS',
    8: 'PASS_COMP',
    9: 'PASS_YDS',
    10: 'PASS_TD',
    13: 'RUSH_YDS',
    14: 'RUSH_TD',
    16: 'REC_YARDS',
    17: 'REC_TD',
    18: 'TARGETS',
    21: 'MISC_TD',
    23: 'WEEK'
}

for index, value in column_mapping.items():
    test_df.columns.values[index] = value


test_df.head(5)

,NAME,POS,TEAM,OPP,@HOME,WON,TEAM_PTS,PROJ_PTS,PASS_COMP,PASS_YDS,PASS_TD,INT,CAR,RUSH_YDS,RUSH_TD,REC,REC_YARDS,REC_TD,TARGETS,2PC,FUML,MISC_TD,FPTS,WEEK
0,Lamar Jackson,QB,Bal,Det,True,True,38,19.1,21,357,3,0,9,36,1,0,0,0,0,0,1,0,33.88,7
1,Patrick Mahomes,QB,KC,LAC,True,True,31,22.8,32,424,4,1,4,29,0,0,0,0,0,0,0,0,33.86,7
2,Gardner Minshew,QB,Ind,Cle,True,False,38,11.5,15,305,2,1,3,29,2,0,0,0,0,0,3,0,27.1,7
3,Josh Allen,QB,Buf,NE,False,False,25,19.7,27,265,2,1,7,17,1,0,0,0,0,0,1,0,24.3,7
4,Jalen Hurts,QB,Phi,Mia,True,True,31,23.5,23,279,2,1,11,21,1,0,0,0,0,0,1,0,23.26,7


In [12]:
oldData.head(5)

,NAME,POS,TEAM,OPP,@HOME,WON,TEAM_PTS,PROJ_PTS,PASS_COMP,PASS_YDS,PASS_TD,INT,CAR,RUSH_YDS,RUSH_TD,REC,REC_YARDS,REC_TD,TARGETS,2PC,FUML,MISC_TD,FPTS,WEEK,DnP
0,Tua Tagovailoa,QB,Mia,LAC,False,True,36,17.6,28,466,3,1,5,5,0,0,0,0,0,0,1,0,27.14,1,False
1,Mac Jones,QB,NE,Phi,True,False,20,14.7,35,316,3,1,2,15,0,0,0,0,0,0,0,0,24.14,1,False
2,Jordan Love,QB,GB,Chi,False,True,38,14.8,15,245,3,0,3,12,0,0,0,0,0,0,0,0,23.00,1,False
3,Anthony Richardson,QB,Ind,Jax,True,False,21,17.2,24,223,1,1,10,40,1,0,0,0,0,0,0,0,20.92,1,False
4,Justin Herbert,QB,LAC,Mia,True,False,34,16.9,23,229,1,0,5,17,1,0,0,0,0,0,0,0,20.86,1,False


In [11]:
oldData = pd.read_csv('WK_1-6.csv')
oldData.drop(columns=['DnP'], inplace=True)
print(oldData.shape)
print(test_df.shape)
# Reset the index of both DataFrames
oldData.reset_index(drop=True, inplace=True)
test_df.reset_index(drop=True, inplace=True)

combined_df = pd.concat([oldData, test_df], ignore_index=True)
print(combined_df.shape)
combined_df.to_csv('WK_1-7.csv', index=False)

(3565, 25)
(507, 24)


InvalidIndexError: Reindexing only valid with uniquely valued Index objects

In [11]:
new_df = pd.read_csv('QB-RB-WR-TE.csv')
columns_to_fill = [
    'POINTS_SCORED', 'PROJ', 'COMP_PASSES', 'PASS_YDS', 'PASS_TD', 'INT',
    'RUSH_CAR', 'RUSH_YDS', 'RUSH_TD', 'RECEPTIONS', 'REC_YDS', 'REC_TD',
    'TAR', '2PC', 'FUML', 'MISC_TD', 'FPTS'
]

# Use fillna to replace NaN values with 0 for the specified columns
new_df[columns_to_fill] = new_df[columns_to_fill].fillna(0)

new_df.to_csv('QB-RB-WR-TE.csv', index=False)

In [2]:
df = pd.read_csv('QB-RB-WR-TE.csv')

In [9]:
pd.set_option('display.max_columns', None)

df.head()

,NAME,POSITION,TEAM,OPP,AT_HOME,WON,POINTS_SCORED,PROJ,COMP_PASSES,PASS_YDS,PASS_TD,INT,RUSH_CAR,RUSH_YDS,RUSH_TD,RECEPTIONS,REC_YDS,REC_TD,TAR,2PC,FUML,MISC_TD,FPTS,WEEK
0,Tua Tagovailoa,QB,Mia,LAC,False,True,36.0,17.6,28,466,3,1,5,5,0,0,0,0,0,0,1,0,27.14,1
1,Mac Jones,QB,NE,Phi,True,False,20.0,14.7,35,316,3,1,2,15,0,0,0,0,0,0,0,0,24.14,1
2,Jordan Love,QB,GB,Chi,False,True,38.0,14.8,15,245,3,0,3,12,0,0,0,0,0,0,0,0,23.00,1
3,Anthony Richardson,QB,Ind,Jax,True,False,21.0,17.2,24,223,1,1,10,40,1,0,0,0,0,0,0,0,20.92,1
4,Justin Herbert,QB,LAC,Mia,True,False,34.0,16.9,23,229,1,0,5,17,1,0,0,0,0,0,0,0,20.86,1
